In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing requirements

In [ ]:
import numpy as np
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import matplotlib.pyplot as plt
import seaborn as sns
import random

from IPython.display import YouTubeVideo

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau

import warnings
warnings.filterwarnings('ignore')

## Getting dataset

In [ ]:
train_df = pd.read_csv("../input/digit-recognizer/train.csv")
test_df = pd.read_csv("../input/digit-recognizer/test.csv")
train_df.shape, test_df.shape

In [ ]:
train_df.head()

## Reshaping Data and spliting

In [ ]:
X = train_df.drop(columns = ['label']).values.astype('float32')
y = train_df['label'].values
print(X)
print(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, stratify = y, random_state = 0)

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28)
X_test = X_test.reshape(X_test.shape[0], 28, 28)

In [ ]:
X_train.shape


In [ ]:
X_test

## Visualizing the Images

In [ ]:
def images_and_pixels(image, ax):
    
    ax.imshow(image, cmap = 'PuBu')
    width, height = image.shape
    threshold = image.max() / 2.5
    
    for x in range(width):
        for y in range(height):
            ax.annotate(str(round(image[x][y],2)), 
                        xy = (y,x),
                        horizontalalignment = 'center',
                        verticalalignment = 'center',
                        color = 'white' if image[x][y] < threshold else 'black')

fig = plt.figure(figsize = (10,10)) 
ax = fig.add_subplot(111)
random_number = random.randint(0, 37800)

images_and_pixels(X_train[1], ax)

plt.title('Label = ' + str(y_train[random_number]), fontsize = 20)
plt.axis('off')
plt.show()


In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

## Data Augmentation

In [ ]:
model = keras.Sequential([
            keras.Input(shape = (28, 28, 1)),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
            layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
            layers.Conv2D(128, 3, padding = 'same', activation = 'relu'),
            layers.Flatten(),
            layers.Dense(128, activation = 'relu'),
            layers.Dense(10, activation = 'softmax'),
])

model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
        optimizer = keras.optimizers.Adam(lr = 1e-4),
        metrics = ['acc'],
)

In [ ]:
model.summary()

### Using Learning Rate Reduction

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_acc', patience = 3, verbose = 1, factor = 0.5, min_lr = 0.00001)

In [ ]:
classifier = model.fit(X_train, y_train, batch_size = 64, 
                                 epochs = 50, 
                                 validation_data = (X_test, y_test), 
                                 verbose = 1,
                                 steps_per_epoch = X_train.shape[0] // 64,
                                 callbacks = [reduce_lr])


## Model performance

In [ ]:
def model_performance_graphs():
    
    fig, axes = plt.subplots(1, 2, figsize = (15, 8))

    axes[0].plot(classifier.epoch, classifier.history['acc'], label = 'acc')
    axes[0].plot(classifier.epoch, classifier.history['val_acc'], label = 'val_acc')
    axes[0].set_title('Accuracy vs Epochs', fontsize = 20)
    axes[0].set_xlabel('Epochs', fontsize = 15)
    axes[0].set_ylabel('Accuracy', fontsize = 15)
    axes[0].legend()

    axes[1].plot(classifier.epoch, classifier.history['loss'], label = 'loss')
    axes[1].plot(classifier.epoch, classifier.history['val_loss'], label="val_loss")
    axes[1].set_title("Loss Curve",fontsize=18)
    axes[1].set_xlabel("Epochs",fontsize=15)
    axes[1].set_ylabel("Loss",fontsize=15)
    axes[1].legend()

    plt.show()
    
model_performance_graphs()


## Model evaluation

In [ ]:
model.evaluate(X_train, y_train, batch_size = 64, verbose = 1)

In [ ]:
model.evaluate(X_test, y_test, batch_size = 64, verbose = 1)

In [ ]:
final_labels = model.predict(((test_df.values).reshape(-1, 28, 28, 1) / 255.0))
final_labels = np.argmax(final_labels, axis = 1)


## Creating the Submission File

In [ ]:
submission_df = pd.DataFrame()
image_id = [i for i in range(1, 28001)]
submission_df['ImageId'] = image_id
submission_df['Label'] = final_labels
submission_df.to_csv('submission.csv', index = False)
submission_df.head(10)
